In [1]:
from PIL import Image
import numpy as np
import os
from pathlib import Path
import logging
from tqdm import tqdm  # For progress tracking

def preprocess_directory_images(directory_path, output_size=(1024, 1024), resize_method='bicubic'):
    """
    Preprocess all images in a directory by resizing them to the specified dimensions.
    Original images are replaced with the processed versions.
    
    Parameters:
        directory_path (str): Path to the directory containing images
        output_size (tuple): Desired output dimensions (width, height)
        resize_method (str): The resampling filter to use
    """
    # Set up logging to track any errors or issues
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    
    # Create a mapping for different image formats that PIL can handle
    supported_formats = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff'}
    
    # Create filter mapping for resize methods
    filter_map = {
        'nearest': Image.NEAREST,
        'bilinear': Image.BILINEAR,
        'bicubic': Image.BICUBIC,
        'lanczos': Image.LANCZOS
    }
    resample_filter = filter_map.get(resize_method.lower(), Image.BICUBIC)
    
    # Get all image files in the directory
    directory = Path(directory_path)
    image_files = [f for f in directory.glob('*') if f.suffix.lower() in supported_formats]
    
    if not image_files:
        logger.warning(f"No supported image files found in {directory_path}")
        return
    
    # Process each image with a progress bar
    for image_path in tqdm(image_files, desc="Processing images"):
        try:
            # Load the image
            img = Image.open(image_path)
            
            # Convert to RGB if image is in another mode
            if img.mode != 'RGB':
                img = img.convert('RGB')
            
            # Calculate aspect ratio preserving dimensions
            original_width, original_height = img.size
            aspect_ratio = original_width / original_height
            
            if aspect_ratio > 1:
                new_width = output_size[0]
                new_height = int(new_width / aspect_ratio)
            else:
                new_height = output_size[1]
                new_width = int(new_height * aspect_ratio)
            
            # Resize the image while preserving aspect ratio
            img_resized = img.resize((new_width, new_height), resample_filter)
            
            # Create a new black background
            background = Image.new('RGB', output_size, (0, 0, 0))
            
            # Calculate position to paste the resized image
            paste_x = (output_size[0] - new_width) // 2
            paste_y = (output_size[1] - new_height) // 2
            
            # Paste the resized image onto the background
            background.paste(img_resized, (paste_x, paste_y))
            
            # Save the processed image, replacing the original
            # Using high quality for JPEG to minimize compression artifacts
            background.save(image_path, quality=95, optimize=True)
            
            logger.info(f"Successfully processed: {image_path.name}")
            
        except Exception as e:
            logger.error(f"Error processing {image_path.name}: {str(e)}")
            continue

# Example usage:
directory_path = 'images/'
preprocess_directory_images(
    directory_path,
    output_size=(1024, 1024),
    resize_method='bicubic'
)

Processing images:   6%|▋         | 3/47 [00:00<00:11,  3.96it/s]INFO:__main__:Successfully processed: pexels-mutecevvil-15679626.jpg
INFO:__main__:Successfully processed: pexels-kelly-1179532-29512751.jpg
Processing images: 100%|██████████| 47/47 [00:13<00:00,  3.46it/s]
